# 라이브러리 설치

In [ ]:
!pip install fitz tools frontend pdfplumber python-dotenv
!mkdir static

# 파일 다운로드

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from dotenv import load_dotenv

# 환경 변수
load_dotenv()

MILKT_ID = os.getenv("MILKT_ID")                    # 아이디
MILKT_PASSWORD = os.getenv("MILKT_PASSWORD")        # 비밀번호
PDF_DOWNLOAD_PATH = os.getenv("PDF_DOWNLOAD_PATH")  # 절대경로

# 밀크티 아이디, 비밀번호 설정 확인
if not MILKT_ID or not MILKT_PASSWORD:
    raise ValueError("MILKT_ID 또는 MILKT_PASSWORD 환경 변수가 설정되지 않았습니다.")

# PDF폴더 확인
if not os.path.exists(PDF_DOWNLOAD_PATH):
    os.makedirs(PDF_DOWNLOAD_PATH,mode=0o777)

# 크롤링 대기 시간
DELAY_DURATION = 0.5

# 셀레니움 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "profile.default_content_settings.popups": 0,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "profile.default_content_setting_values.automatic_downloads": 1,  # 자동 다운로드 허용
    "download.default_directory": PDF_DOWNLOAD_PATH
})

# 필요시 헤드리스 모드 설정
# options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

try:
    # 로그인 페이지 접근 및 로그인 처리
    driver.get("https://www.milkt.co.kr/Login/Login")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'input_id')))
    driver.find_element(By.ID, 'input_id').send_keys(MILKT_ID)  # 아이디
    driver.find_element(By.ID, 'input_pw').send_keys(MILKT_PASSWORD)  # 비번
    driver.find_element(By.ID, 'btn_login').click()
    # time.sleep(DELAY_DURATION)  # 로그인 후 로딩 대기

    # 팝업 창 닫기 처리
    main_window = driver.current_window_handle
    for handle in driver.window_handles:
        if handle != main_window:
            driver.switch_to.window(handle)
            time.sleep(DELAY_DURATION)
            driver.close()
            driver.switch_to.window(main_window)

    try:
        close_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "btn_close"))
        )
        close_button.click()
        print("레이어 팝업이 닫혔습니다.")
    except:
        print("레이어 팝업이 없습니다.")

    # 페이지 접속
    url = f"https://www.milkt.co.kr/LectureMaterial/LectureMaterial?intPageSize=30&strCode=A04&strSubCode=A0402&inGrade=1&inTerm=1&strlectTit=&strlectCont="
    driver.get(url)

    # "전체 선택" 체크 박스 체크
    try:
        label = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//label[@for='aAll']"))
        )
        label.click()  # Select all items
        print(f"페이지에서 전체 선택을 완료했습니다.")

        # 다운로드 버튼 찾기
        download_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'on') and text()='다운로드']"))
        )
        # 다운로드 버튼 클릭
        download_button.click()  
        print(f"페이지에서 다운로드 버튼을 클릭했습니다.")

        time.sleep(DELAY_DURATION)  # 대기 시간 (서버가 처리할 시간을 주기 위해)

        # 다운로드 확인창 대기 후 확인
        alert = WebDriverWait(driver, 5).until(EC.alert_is_present())
        alert.accept()  # Click OK on the confirmation dialog
        print(f"페이지에서 다운로드 확인 창을 확인했습니다.")

        time.sleep(5)  # 다운로드가 실제로 완료되도록 더 긴 대기 시간 추가
        
        
    except Exception as e:
        print(f"페이지에서 오류가 발생했습니다: {e}")

finally:
    # 드라이버 종료
    driver.quit()


레이어 팝업이 닫혔습니다.
페이지에서 전체 선택을 완료했습니다.
페이지에서 다운로드 버튼을 클릭했습니다.
페이지에서 다운로드 확인 창을 확인했습니다.
